In [ ]:
#!/usr/bin/env python3
"""
    File Name: 4D Dataset Generator.ipynb
    Author: Thomas Frew
    Date created: 07/01/2023
    Date last modified: 24/01/2023
    Python Version: 3.11
    
    Read PNGs from a specified directory. The user can label each
    image with the top-left and bottom-right points of a bounding
    box, enclosing some object of interest.
    
    Each file name is saved in a CSV with its respective (4-dimensional)
    bounding box data.
"""

In [ ]:
# Use the tkinter backend for Matplotlib
%matplotlib tk

# Import Matplotlib functions
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Import helper functions
import helpers

In [ ]:
def displayImage(file_names, i):
    """
    Display the ith image from a list of PNG file names, into the interface
    """
    
    # If there is an ith file...
    if (i < len(file_names)):
        
        # Read the file's data
        image_data = mpimg.imread(file_names[i])
        
        # Plot and display the file
        imgplot.set_data(image_data)
        fig.canvas.draw_idle()

In [ ]:
# Click-reading interactivity
def labelImage(event):
    """
    Read a click from the dataset-generating UI and write its co-ordinate data to the training dataset.
    """
    global i, completion
    
    # If there are still images to label...
    if (i < len(file_names)):
        
        # Read the coordinates of the user's click
        x, y = int(event.xdata), int(event.ydata)

        # If the click was outside the image, warn the user and do nothing
        if (x == None or y == None):
            print("No object found")
                
        # If we have not recorded the top-left point of each image's bounding box...
        elif (not completion[0]):
            
            # Create a new dataset entry, with the current file name and top-left point
            data = [file_names[i],x,y]
            training_data.append(data)
            print(f"{file_names[i]}: {x},{y}")
            
        # Otherwise, this click was for the bottom-right point...
        else:
            
            # Append the bottom-right point to the existing dataset entry
            training_data[i].extend([x,y])
            print(f"{training_data[i][0]}: {training_data[i][1]},{training_data[i][2]} {x},{y}")
            
        # Display the next image
        i += 1
        displayImage(file_names,i)

    # If there are no more images to label...
    if (i == len(file_names)):
    
        # Otherwise, if we not recorded the top-left point of each image's bounding box...
        if (not completion[0]):
            
            # Store that we have recorded the top-left point of each image's bounding box
            completion[0] = True
            
            # Report that the user must now click on the bottom-right point of each image's bounding box
            plt.title("Bottom-Right Points")
            print("You have recorded the top-left point of each image's bounding box.")
            print("Now, please click on the bottom-right point of each image's bounding box.")
            
            # Reset the index of the current image, so we can begin storing bottom-right points
            i = 0
            displayImage(file_names,i)

        # Otherwise, if we have not recorded the bottom-right point of each image's bounding box
        elif (not completion[1]):

            # Store that we have recorded the bottom-right point of each image's bounding box
            completion[1] = True
            plt.title("Dataset Complete")

            # Save the training data
            helpers.save_training_dataset(training_data)

            # Report that the training data saved successfully
            print("Dataset saved successfully.")

In [ ]:
# Create global, dataset-management variables
i = 0
training_data = []
completion = [False, False]

In [ ]:
# Read and store the dataset
directory = helpers.read_dataset_dir()
file_names = helpers.read_pngs(directory)

In [ ]:
# Produce the interface to record point data
fig = plt.figure("4D Dataset Generator", figsize=(7,7))

# Report that the user must click on the top-left of each image's bounding box
plt.title("Top-Left Points")
print("Please click on the top-left point of each image's bounding box.")

In [ ]:
# Display the first image for the user to label
image_data = mpimg.imread(file_names[0])
imgplot = plt.imshow(image_data)

# Establish an event associated with clicking the figure (labelImage)
click_id = fig.canvas.mpl_connect('button_press_event', labelImage)